# 加载库函数及导入文件

In [55]:
import pandas as pd
import pathlib
base_path = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_1_1赵益华.xlsx")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
output_dir_path = pathlib.Path(r"E:\张楠\测试\output")
%run corelibs.py
print(base_path)
print(acc_file)
print(output_dir_path)

E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人
E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls
E:\张楠\测试\output


In [21]:
import yaml
with open('config.yaml', 'r', encoding='utf-8') as f:
    CONF_DATA = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
CONF_DATA['天津银行']['账户']

# 测试

In [ ]:
if base_path.is_dir():
    for  _p in base_path.rglob('*.xlsx'):
        print(_p)

##  测试单个账户文件读取及储存

In [56]:
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
df = parse_sheet_general(acc_file,0,CONF_DATA['天津银行']['账户'])

In [ ]:
df.info()

In [ ]:
df

In [59]:
save_accounts(df, output_dir_path, '天津银行')

4

## 测试单个流水文件的读取与储存

In [9]:
grouped = df.groupby('账号')
df_list = [x.reset_index(drop=True) for _ , x in grouped]
save_statements(df_list, output_dir_path, '天津银行',"津滨查询(2023)138号")

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:07<00:00,  4.71it/s]


13162

In [20]:
verify_data(df, CONF_DATA['天津银行']['账户']['verify_cols'])

False

In [ ]:
df_list

In [ ]:
pd.read_excel

# 执行

In [ ]:
df = excel_file.parse(sheet_name=0, header=0, skiprows=0, nrows=5, dtype=str, converters={'交易金额':float})
df.info()

In [ ]:
pd.to_datetime(df['交易日期']).dt.time

In [85]:
converters = {_col:float for _col in CONF_DATA['天津银行']['流水']['amount_cols']}
converters

{'交易金额': float, '余额': float}

# 杂项

In [45]:
base_path

WindowsPath('E:/张楠/2023赵益华927/银行/天津银行/滨监查询（2023）05217.xls')

In [37]:
a = {1:2}
next(iter(a))

1

In [38]:
_new_cols = {} # 构造需要填充统一内容的列字典，通常是新列
_verify_cols = {} # 构造需要进行数据检查的列字典，（v1）版本只检查空值
_col_name_map = {} # 构造需要改名的列字典
_merge_2cols = {} # 构造需要合并的列字典

#根据配置信息生成列处理逻辑
for _key, _val in CONF_DATA['天津银行']['账户'].items(): # 读取配置中的列名（key）和列配置（val）
    if isinstance(_val, str): # 如果列配置为str，则本列全部内容为该字符串，通常是全新的列
        _new_cols[_key] = _val
    elif isinstance(_val, bool) and _val: # 如果列配置为bool且为True，代表本列需要数据检查
        _verify_cols[_key] = True # （v1）版本只检查空值，此处恒为True
    elif isinstance(_val, dict): # 如果列配置为dict，则分情况讨论
        for __k, __v in _val.items(): # 首先检查所有字典值，将为True的加入数据检查中
            if __v:
                 _verify_cols[__k] = True
        if len(_val) == 1: # 当字典中只有一项时，代表本列需要改名
            _col_name_map[next(iter(_val))] = _key
        elif len(_val) > 1: # 当字典中有两项或更多时，代表本列需要合并多个原始列（目前仅支持两个）
            _merge_2cols[_key] = list(_val.keys())
        else:
            raise Exception(f"字典类型长度错误，检查配置列：{_key}") 
    elif isinstance(_val, list): # 
        if _key != 'cols_new_order':
            raise Exception(f"只有cols_new_order可以配置为list类型，检查配置列：{_key}")                 
    else:
        raise Exception(f"配置内容类型不支持，检查配置列：{_key}") 


print(_new_cols)
print(_verify_cols)
print(_col_name_map)
print(_merge_2cols)


{'银行': '天津银行'}
{'开户名称': True, '证件号码': True}
{'开户名称': '姓名', '客户账号': '账号', '开户机构名称': '开户机构', '销户机构名称': '销户机构'}
{'电话': ['短信捆绑电话', '网银电话']}
